In [1]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
import cv2
import numpy as np
import time

In [4]:
p = [(n1, n2) for n1 in range(2) for n2 in range(6,8)]

print (p)

[num**2 if num %2 ==0 else 0 for num in range(10)]

[(0, 6), (0, 7), (1, 6), (1, 7)]


[0, 0, 4, 0, 16, 0, 36, 0, 64, 0]

In [2]:
it

In [2]:
# th architecture to use
arch = 'resnet50'
# load the pre-trained weights
model_file = './models/%s_places365.pth.tar' % arch
model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# load the class label
file_name = './categories/categories_places365.txt'
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [4]:
# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
cap = cv2.VideoCapture('.\\imgs\\3.MOV')  
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
frames = 0
lblDic = {}
start = time.time() 
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.resize(frame,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    cv2_im = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2_im)
   
    input_img = V(centre_crop(img).unsqueeze(0))
    # forward pass
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    preds = ''
    horPos = 10
    verPos = 50
    label = []
    # output the prediction
    for i in range(0, 5):
        preds = "{} = {:.3f}".format(classes[idx[i]],probs[i])
        label.append({classes[idx[i]]:"{:.3f}".format(probs[i])})
        cv2.putText(frame,preds,(horPos,verPos), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
        verPos += 30
    print(label)
    lblDic[frames] = label
    frames += 1
    print ("Frames : "+ str(frames))
    print("FPS of the video is {:5.2f}".format( frames / (time.time() - start)))
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[{'jail_cell': '0.590'}, {'sauna': '0.162'}, {'barndoor': '0.041'}, {'elevator/door': '0.039'}, {'staircase': '0.018'}]
Frames : 1
FPS of the video is  7.16
[{'jail_cell': '0.606'}, {'sauna': '0.146'}, {'barndoor': '0.043'}, {'elevator/door': '0.040'}, {'basement': '0.018'}]
Frames : 2
FPS of the video is  7.32
[{'jail_cell': '0.636'}, {'sauna': '0.127'}, {'barndoor': '0.037'}, {'elevator/door': '0.036'}, {'staircase': '0.019'}]
Frames : 3
FPS of the video is  7.46
[{'jail_cell': '0.658'}, {'sauna': '0.107'}, {'barndoor': '0.044'}, {'elevator/door': '0.025'}, {'staircase': '0.023'}]
Frames : 4
FPS of the video is  7.88
[{'jail_cell': '0.693'}, {'sauna': '0.077'}, {'elevator/door': '0.030'}, {'staircase': '0.029'}, {'barndoor': '0.021'}]
Frames : 5
FPS of the video is  8.02
[{'jail_cell': '0.621'}, {'sauna': '0.072'}, {'staircase': '0.049'}, {'elevator/door': '0.029'}, {'stable': '0.027'}]
Frames : 6
FPS of the video is  8.22
[{'jail_cell': '0.697'}, {'sauna': '0.109'}, {'staircase': '0

In [6]:
import json
with open('places.json', 'w') as outfile:
    json.dump(lblDic, outfile)

NameError: name 'lblDic' is not defined

In [13]:
# load the test image

img_name = './imgs/2.JPG'
img = Image.open(img_name)
input_img = V(centre_crop(img).unsqueeze(0))
# forward pass
logit = model.forward(input_img)
h_x = F.softmax(logit, 1).data.squeeze()
probs, idx = h_x.sort(0, True)

print('{} prediction on {}'.format(arch,img_name))
# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

resnet50 prediction on ./imgs/2.JPG
0.462 -> kitchen
0.281 -> utility_room
0.086 -> galley
0.048 -> laundromat
0.035 -> wet_bar


In [11]:
print (classes[idx[0]])

kitchen
